# Iterator workflow for cables
To be able to use the Iterator, the following steps need to be followed to install py-ofx on your laptop:
1. Clone the py-ofx repository to your laptop: https://github.com/VanOord/py-ofx;
2. Check out the temp-mixin-fix branch;
3. Install py-ofx by running `pip install -e .` in the top level of the repository.

After following those steps, you should be able to perform the following imports:

In [1]:
import OrcFxAPI as ofx
from py_ofx.utils import Iterator

The general procedure for working with the Iterator object is as follows:
0. Create example model (for this session only);
1. Prepare your model;
2. Load model file into Iterator;
3. Load checker functions;
4. Define iteration parameter;
5. Run.

## 0. Create example model
For this session we create a model using python so that no *dat file needs to be loaded.

In [2]:
# Create model
model = ofx.Model()

# Add line object
line = model.CreateObject(ofx.otLine, name="Cable")

## 1. Prepare your model
In principle this is done in the same way as in the previous workflow. The parameters that do not change during the iteration should be all set in advance.

In [3]:
# Set JONSWAP waves
model.environment.WaveType = "JONSWAP"

# Set wave parameters
model.environment.WaveTp = 8

# Set simulation duration
model.general.StageCount = 2
model.general.StageDuration = [10, 10800]

## 2. Load model into Iterator
Create an Iterator object and load the Orcaflex model into it. For demonstration purposes we set the time interval for checking if the limits are exceeded to 30 seconds.

In [4]:
it = Iterator(model)
it.limit_check_interval = 30

## 3. Load checker functions
Here the workflow starts deviating significantly from the previous workflow. To let the Iterator object check the limiting criteria, checker functions must be defined and loaded into the object. An example of a simple checker function looks like this:
```
def checker_function(model, period):
    data = model["Cable"].TimeHistory("Effective tension", period, ofx.oeEndA)
    exceeded = data.max() > 42
    return (exceeded, data)
```
The reason this has to be in the form of a function is that the Iterator will change the variable period every time it calls this function so that the checks are done for example at the following periods:
- from start to 0 seconds
- from 0 to 5 seconds
- from 5 to 10 seconds
- from 10 seconds until the end

If the first argument which the checker function returns is True, the simulation will be aborted and restarted after re-setting the iteration parameter.

The second argument that the checker function returns is the data from the TimeHistory or RangeGraph, which can be used for post-processing purposes. This avoids the need to write the same Orcaflex code twice.

Let's use the example checker function above in this model:

In [5]:
def checker_function(model, period):
    data = model["Cable"].TimeHistory("Effective tension", period, ofx.oeEndA)
    exceeded = data.max() > 42
    return (exceeded, data)

Now the checker function can be loaded into the Iterator:

In [6]:
it.set_checker_funcs(checker_function)

## 4. Define iteration parameter
Normally we use the significant wave height as iteration parameter. With this function you can set any Orcaflex parameter as iteration parameter. Note that the parameter should be written exactly how Orcaflex defines the parameter, such as WaveHs, WaveHeight etc.

Currently the Iterator can only iterate down, so the step must be negative.

In [7]:
it.set_iteration_parameter(parameter="WaveHs", start=1.5, end=0, step=-0.25)

## 5. Run

In [8]:
it.run()

Iteration parameter: WaveHs, start value: 2.5
Running simulation, WaveHs = 2.5
Simulation aborted at 5260.4s, limits exceeded function checker_function

Running simulation, WaveHs = 2.25
Simulation aborted at 5227.2s, limits exceeded function checker_function

Running simulation, WaveHs = 2.0
Simulation aborted at 5124.9s, limits exceeded function checker_function

Running simulation, WaveHs = 1.75
Simulation aborted at 5063.6s, limits exceeded function checker_function

Running simulation, WaveHs = 1.5
Simulation aborted at 4979.9s, limits exceeded function checker_function

Running simulation, WaveHs = 1.25
Simulation aborted at 4736.6s, limits exceeded function checker_function

Running simulation, WaveHs = 1.0
Simulation passed, WaveHs = 1.0

